In [15]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import numpy as np
import scipy as sp
import scipy.integrate
import scipy.optimize
output_notebook()

Loading BokehJS ...

# No drag

The equations of motion of a projectile fired by a cannon is given by:
$$
  \frac{d^2 \vec{r}}{d t^2} = -g \vec{e}_y
$$
The value for $g$ is $9.81 m/s^2$, and the initial velocity $v_0$ is $700 m/s$.

In [5]:
g = 9.81

## Equations

As usual, the second order differential equation is transformed into a set of first order equations, and the right hand side of the equations is defined by the function `rhs`.  Additionally, the Jacobian is provided.

In [6]:
def rhs(t, Y, g):
    x, y, v_x, v_y = Y[0], Y[1], Y[2], Y[3]
    v = np.sqrt(v_x**2 + v_y**2)
    return np.array([
        v_x,
        v_y,
        0.0,
        -g,
    ])

In [7]:
def jac(t, Y, g):
    x, y, v_x, v_y = Y[0], Y[1], Y[2], Y[3]
    v = np.sqrt(v_x**2 + v_y**2)
    return np.array([
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0],
    ])


Since it is more convenient to approach the problem using degrees rather than radians, a function is defined to compute the initial values.

In [18]:
def compute_init_values(alpha, v_0 = 700):
    alpha_rad = np.pi*alpha/180.0
    return np.array([0.0, 0.0,
                     v_0*np.cos(alpha_rad), v_0*np.sin(alpha_rad),
                     ])

## Solution

We can solve the set of equations for an angle of 40 degrees.

In [20]:
init_values = compute_init_values(40.0)
x_0, y_0, v_x_0, v_y_0 = init_values

Define the ODE system, setting initial conditions and parameters.

In [10]:
sys = scipy.integrate.ode(rhs, jac) \
           .set_integrator('dopri5') \
           .set_initial_value(init_values, 0.0) \
           .set_f_params(g) \
           .set_jac_params(g)

In [11]:
delta_t = 1.0e-3

In [12]:
t, x, y, v_x, v_y = [0.0], [x_0], [y_0], [v_x_0], [v_y_0]

In [13]:
while sys.successful() and y[-1] >= 0:
    sys.integrate(sys.t + delta_t)
    t.append(sys.t)
    x.append(sys.y[0])
    y.append(sys.y[1])
    v_x.append(sys.y[2])
    v_y.append(sys.y[3])

In [38]:
fig = figure(plot_width=500, plot_height=300,
             x_axis_label='x', y_axis_label='y')
fig.line(x, y)
show(fig)

## Range as a function of the firing angle

In [25]:
def compute_range(alpha, delta_t = 1.0e-3):
    init_values = compute_init_values(alpha)
    x_0, y_0, v_x_0, v_y_0 = init_values
    sys = scipy.integrate.ode(rhs, jac) \
               .set_integrator('dopri5') \
               .set_initial_value(init_values, 0.0) \
               .set_f_params(g) \
               .set_jac_params(g)
    t, x, y, v_x, v_y = [0.0], [x_0], [y_0], [v_x_0], [v_y_0]
    while sys.successful() and y[-1] >= 0:
        sys.integrate(sys.t + delta_t)
        t.append(sys.t)
        x.append(sys.y[0])
        y.append(sys.y[1])
        v_x.append(sys.y[2])
        v_y.append(sys.y[3])
    return 0.5*(x[-2] + x[-1])

In [26]:
alphas = np.linspace(10.0, 80.0, 50)
ranges = [compute_range(alpha) for alpha in alphas]

In [27]:
fig = figure(plot_width=500, plot_height=300,
             x_axis_label='alpha', y_axis_label='range')
fig.line(alphas, ranges)
show(fig)

## Maximal range

In [29]:
def helper_func(alpha):
    return -compute_range(alpha)

In [37]:
scipy.optimize.minimize_scalar(helper_func, bracket=(10.0, 80.0),
                               method='golden')

     fun: -49949.26057595005
    nfev: 44
     nit: 39
 success: True
       x: 44.97543056302328

# Drag

The equations of motion of a projectile fired by a cannon is given by:
$$
  \frac{d^2 \vec{r}}{d t^2} = -g \vec{e}_y - \frac{B_2}{m} v e^{-\frac{y}{y_d}} \vec{v} 
$$
Realistic values for $g = 9.81 \frac{m}{s^2}$, $\frac{B_2}{m} = 4\cdot10^{-5} m^{-1}$, $y_d = 10^4 m$.

In [2]:
def rhs(t, Y, g, b_2_m, y_d):
    x, y, v_x, v_y = Y
    v = np.sqrt(v_x**2 + v_y**2)
    return [
        v_x,
        v_y,
        -b_2_m*v*np.exp(-y/y_d)*v_x,
        -g - b_2_m*v*np.exp(-y/y_d)*v_y,
    ]